ДЛЯ КОРРЕКТНОЙ РАБОТЫ ПАРСЕРА НУЖНО ДОБАВИТЬ АКТУАЛЬНЫЕ ССЫЛКИ В ФАЙЛ links.csv В СТОЛБЕЦ "Ссылка"

In [40]:
import requests
import csv
from fake_useragent import UserAgent
from datetime import date
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import warnings
import re 

# from selenium.webdriver.firefox.options import Options 
# import concurrent.futures

warnings.filterwarnings('ignore')

In [41]:
def make_request(link):
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    try:
        url = link
        response = requests.get(url, headers=headers, timeout=30, verify=False)
        parsed = response.json()
        return parsed
    except requests.exceptions.ConnectionError:
        print('Connection error!!!')


def wb_parser(link):
    link = link.replace('https://www.wildberries.ru/catalog/', '')
    if '/' in link:
        index = link.find('/')
        link = link.replace(link[index:], '')
    # try:
    #     parsed = make_request(f"https://card.wb.ru/cards/detail?appType=1&curr=rub&dest=-1257786&regions=80,83,38,4,64,33,68,70,30,40,86,75,69,1,66,110,22,48,31,71,112,114&spp=26&nm={link}")
    # except:
    parsed = make_request(f"https://card.wb.ru/cards/v1/detail?appType=1&curr=rub&dest=-1570684&spp=27&nm={link}")
    try:
        sizes = ['17.5', '17', '18', '19', '45', '50']
        for i in range(len(parsed['data']['products'][0]['sizes'])):
            size = parsed['data']['products'][0]['sizes'][i]['origName']
            if size in sizes:
                price = parsed['data']['products'][0]['sizes'][i]['salePriceU'] / 100
                break
    except:
        try:
            for i in range(len(parsed['data']['products'])):
                if parsed['data']['products'][i]['id'] == int(link):
                    price = parsed['data']['products'][i]['salePriceU'] / 100
                    break
        except:
            price = np.nan           
    discount = 1
    product = [price, discount]
    return product       
    

In [42]:
def sunlight_parser(link):
    for i in link:
        try:
            int(i)
            id = link.index(i)
            link = link[id:]
            id_ = link.index('.')
            link = link[:id_]
            break
        except:
            continue

    cookies = {
        'city_id': '118',
        'city_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
        'city_full_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
        'region_id': '090a0c31-3b91-49a3-aa35-0c784898170b',
        'region_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
        'region_subdomain': 'spb',
        'tmr_lvid': '5f6ab165f41caef626387d8e9d8799df',
        'tmr_lvidTS': '1697011590764',
        '_ym_uid': '1697011591885937639',
        '_ym_d': '1697011591',
        'qr-go-to-the-app-banner': '1',
        '_tt_enable_cookie': '1',
        '_ttp': 'fKiEqa8XyglrSfN_aUZvbCNG6ew',
        'ccart': 'off',
        'uxs_uid': '1766f130-680d-11ee-8541-1b4e2f56831c',
        '_gid': 'GA1.2.1640259099.1697631825',
        'city_auto_popup_shown': '1',
        '_ym_isad': '2',
        'session_id': 'e85480a6-9ba8-4772-8afa-57a70a2eafff',
        '_gat_UA-11277336-11': '1',
        '_gat_UA-11277336-12': '1',
        '_ga_HJNSJ6NG5J': 'GS1.1.1697719835.7.0.1697719835.60.0.0',
        'digi_uc': 'W1sidiIsIjg4NDExIiwxNjk3NzE5ODM1OTE5XSxbInYiLCI3MTM4OSIsMTY5NzcwNjEzNzY3Ml0sWyJ2IiwiMjc1NjE1IiwxNjk3NzA2MTEyODk0XSxbInYiLCIzNDI5MDIiLDE2OTc3MDI0ODkyMDVdLFsidiIsIjI3MzkxOSIsMTY5NzcwMjQ3NTU1MV0sWyJ2IiwiMTUyNjA5IiwxNjk3NzAyNDAwMTcwXSxbInYiLCI1MzUzNiIsMTY5NzcwMjM1ODE2NF0sWyJ2IiwiMTUyNjEyIiwxNjk3NzAyMzEzNzI5XSxbInYiLCIyNjI1MzkiLDE2OTc3MDIyOTc4NTRdLFsidiIsIjE1MTk5NiIsMTY5NzcwMjI3ODU1NF1d',
        '_ga': 'GA1.2.1274284271.1697011591',
        '_ga_GRSMPDQ6SE': 'GS1.2.1697719836.7.0.1697719836.60.0.0',
        '_ga_M9ZHPEJXBM': 'GS1.2.1697719836.7.0.1697719836.60.0.0',
        '_ym_visorc': 'b',
        'mindboxDeviceUUID': '6e9424e5-a80e-43b6-b480-b1d86e4084fd',
        'directCrm-session': '%7B%22deviceGuid%22%3A%226e9424e5-a80e-43b6-b480-b1d86e4084fd%22%7D',
    }
    headers = {
        'AB-TESTS': '{"personal_feed":"cumulative"}',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/json',
        # 'Cookie': 'city_id=118; city_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; city_full_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_id=090a0c31-3b91-49a3-aa35-0c784898170b; region_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_subdomain=spb; tmr_lvid=5f6ab165f41caef626387d8e9d8799df; tmr_lvidTS=1697011590764; _ym_uid=1697011591885937639; _ym_d=1697011591; qr-go-to-the-app-banner=1; _tt_enable_cookie=1; _ttp=fKiEqa8XyglrSfN_aUZvbCNG6ew; ccart=off; uxs_uid=1766f130-680d-11ee-8541-1b4e2f56831c; _gid=GA1.2.1640259099.1697631825; city_auto_popup_shown=1; _ym_isad=2; session_id=e85480a6-9ba8-4772-8afa-57a70a2eafff; _gat_UA-11277336-11=1; _gat_UA-11277336-12=1; _ga_HJNSJ6NG5J=GS1.1.1697719835.7.0.1697719835.60.0.0; digi_uc=W1sidiIsIjg4NDExIiwxNjk3NzE5ODM1OTE5XSxbInYiLCI3MTM4OSIsMTY5NzcwNjEzNzY3Ml0sWyJ2IiwiMjc1NjE1IiwxNjk3NzA2MTEyODk0XSxbInYiLCIzNDI5MDIiLDE2OTc3MDI0ODkyMDVdLFsidiIsIjI3MzkxOSIsMTY5NzcwMjQ3NTU1MV0sWyJ2IiwiMTUyNjA5IiwxNjk3NzAyNDAwMTcwXSxbInYiLCI1MzUzNiIsMTY5NzcwMjM1ODE2NF0sWyJ2IiwiMTUyNjEyIiwxNjk3NzAyMzEzNzI5XSxbInYiLCIyNjI1MzkiLDE2OTc3MDIyOTc4NTRdLFsidiIsIjE1MTk5NiIsMTY5NzcwMjI3ODU1NF1d; _ga=GA1.2.1274284271.1697011591; _ga_GRSMPDQ6SE=GS1.2.1697719836.7.0.1697719836.60.0.0; _ga_M9ZHPEJXBM=GS1.2.1697719836.7.0.1697719836.60.0.0; _ym_visorc=b; mindboxDeviceUUID=6e9424e5-a80e-43b6-b480-b1d86e4084fd; directCrm-session=%7B%22deviceGuid%22%3A%226e9424e5-a80e-43b6-b480-b1d86e4084fd%22%7D',
        'Origin': 'https://spb.sunlight.net',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
        'X-Requested-With': 'SunlightFrontendApp',
        'sec-ch-ua': '"Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }
    params = {
        'article': f'{link}',
    }
    
    params1 = {
    'article': f'{link}',
    'city_id': '118',
    }   

    response1 = requests.get(
        'https://api.sunlight.net/v6/catalog/products/card/purchase/',
        params=params1,
        cookies=cookies,
        headers=headers,
    )

    response = requests.get('https://api.sunlight.net/v6/catalog/products/card/prices/',
        params=params,
        cookies=cookies,
        headers=headers,
    )
    discounts = response1.json()
    data = response.json()
    if data['content']['default_price']['base_sum'] == "":
        price = data['content']['default_price']['result_sum']
        discount = 1
        product = [price, discount]
        return product    
    try:
        price = data['content']['sizes']['17.50']['result_sum']
        discount = discounts['content']['sizes']['17.50']['nameplates'][0]['text']['label']
    except:
        try:
            price = data['content']['sizes']['17.00']['result_sum']
            discount = discounts['content']['sizes']['17.00']['nameplates'][0]['text']['label']
        except:
            try:
                price = data['content']['sizes']['19.00']['result_sum']
                discount = discounts['content']['sizes']['19.00']['nameplates'][0]['text']['label']
            except:
                try:
                    price = data['content']['sizes']['45.00']['result_sum']
                    discount = discounts['content']['sizes']['45.00']['nameplates'][0]['text']['label']
                except:
                    try:
                        price = data['content']['sizes']['50.00']['result_sum']
                        discount = discounts['content']['sizes']['50.00']['nameplates'][0]['text']['label']
                    except:
                        try:
                            price = data['content']['sizes']['18.00']['result_sum']
                            discount = discounts['content']['sizes']['18.00']['nameplates'][0]['text']['label']
                        except:
                            try:
                                price = data['content']['sizes']['0.00']['result_sum']
                                discount = discounts['content']['sizes']['0.00']['nameplates'][0]['text']['label']
                            except:
                                price = np.nan
                                discount = np.nan
    price = price.replace('₽', '')
    nums = re.findall(r'\d+', discount)
    nums = [int(i) for i in nums]
    discount = nums[0]
    discount =  1 - discount / 100
    product = [price, discount]
    return product


In [43]:
def sokolov_parser(link):
    cookies = {
    '_userGUID': '0:lnvpvltk:3jPTv4pojH2Pph6FbhhwQgSmBuQf0U3y',
    'guid_city': '0c5b2444-70a0-4932-980c-b4dc0d3f02b5',
    'name_city': '%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0',
    'guid_region': '0c5b2444-70a0-4932-980c-b4dc0d3f02b5',
    'guid_country': '8aa15da9-92a4-4530-ab74-1992c973c539',
    'region_timezone': 'UTC%2B3%3A00',
    'ab-test-user-id': '63cd24c2a743bb2433f8110f500a594d',
    'fuser_id': 'fa4970464c86b53f66816e5840afc71b5c98b495c898a2314d76b2aaf748bba8a%3A2%3A%7Bi%3A0%3Bs%3A8%3A%22fuser_id%22%3Bi%3A1%3Bs%3A32%3A%22fcb2d248286d2f75ced78b87a0bbd8f2%22%3B%7D',
    '_gcl_au': '1.1.1753721034.1697702901',
    '_ym_uid': '1697702901888153736',
    '_ym_d': '1697702901',
    'flocktory-uuid': '88979721-0c4b-427b-87fb-a99179dc480c-4',
    'tmr_lvid': 'ce6c20b9a094b1708f0b7d19879d76d2',
    'tmr_lvidTS': '1697702901032',
    'mindboxDeviceUUID': '6e9424e5-a80e-43b6-b480-b1d86e4084fd',
    'directCrm-session': '%7B%22deviceGuid%22%3A%226e9424e5-a80e-43b6-b480-b1d86e4084fd%22%7D',
    'syte_uuid': 'ab6463f0-6e56-11ee-b1f4-bd5f3a332347',
    'inova_p_sid': 'uek8p4e_231019010820',
    'adrcid': 'ANIHYs14bKTowSR_pRsEFrg',
    'popmechanic_sbjs_migrations': 'popmechanic_1418474375998%3D1%7C%7C%7C1471519752600%3D1%7C%7C%7C1471519752605%3D1',
    'uxs_uid': 'ad56e520-6e56-11ee-a612-3bba5efe5043',
    'syte_ab_tests': '{}',
    '_ga_VRRLSXGS0X': 'GS1.1.1698059152.1.1.1698060661.0.0.0',
    '_ga': 'GA1.1.230823214.1697702901',
    '_ga_WPBGN0ND17': 'GS1.1.1698059120.2.1.1698062200.0.0.0',
    '_ga_MZNG6JMV9K': 'GS1.1.1698059120.2.1.1698062200.60.0.0',
    'theme-mode': 'light',
    'dSesn': '9ed54e7f-1a02-406e-188a-f57f930def32',
    '_dvs': '0:lo4cg9ex:2B3fhga~i67nWJXdMzKuhpi4fxfBt9rR',
    'PHPSESSID': 'n4ou7vmt8b9dcvrf822gobogtj',
    '_csrf': 'ed5f03a44e260703e1fc2576238deadde4595ddafd7fb8db600fdc5ce7a8432fa%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22GpLSO1iEi7wleXIx4gqk1AIL5RSY7R_d%22%3B%7D',
    }

    headers = {
    'authority': 'catalog.sokolov.ru',
    'accept': 'application/json',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'content-type': 'application/json',
    'origin': 'https://sokolov.ru',
    'referer': 'https://sokolov.ru/jewelry-catalog/product/94010432/',
    'sec-ch-ua': '"Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
    'x-city-uuid': '0c5b2444-70a0-4932-980c-b4dc0d3f02b5',
    'x-country-uuid': '8aa15da9-92a4-4530-ab74-1992c973c539',
    'x-region-uuid': '0c5b2444-70a0-4932-980c-b4dc0d3f02b5',
    'x-source': 'site',
    }      
    link = link.replace('https://sokolov.ru/jewelry-catalog/product/', '')
    link = link.replace('/', '')
    response = requests.get(f'https://catalog.sokolov.ru/api/v2/products/{link}/', headers=headers, timeout=30, verify=False)
    product = response.json()
    url = product['url_code']
    time.sleep(0.5)
    item = requests.get(f'https://sokolov.ru/api/v4/catalog/product-info/{url}/', headers=headers, timeout=30, verify=False, cookies=cookies)
    item = item.json()
    try:
        price = item['price'][1]['value']
    except:
        price = item['price'][0]['value']
    time.sleep(0.5)
    item = requests.get(f'https://sokolov.ru/api/v5/products/coupon/{url}/', headers=headers, timeout=30, verify=False)
    item = item.json()
    try:
        discount = (100 - item['sale_value']) / 100
    except:
        discount = 1
    product = [price, discount]
    return product


In [44]:
def ecom_parser(link):
    link = re.findall(r'\d+', link)
    link = [int(i) for i in link]
    link = link[1]
    df_current = df.loc[(df['article'] == link)]
    try:
        price = df_current[df_current['size'] == 17]['discount_price'].mean()
    except:
        try:
            price = df_current[df_current['size'] == 17.5]['discount_price'].mean()
        except:
            try:
                price = df_current[df_current['size'] == 18]['discount_price'].mean()
            except:
                try:
                    price = df_current[df_current['size'] == 19]['discount_price'].mean()
                except:
                    try:
                        price = df_current[df_current['size'] == 45]['discount_price'].mean()
                    except:
                        try:
                            price = df_current[df_current['size'] == 50]['discount_price'].mean()
                        except:
                            price = np.nan
    try:
        discount = df_current['label_template'].value_counts().index.tolist()
        discount = discount[0]
        discount = re.findall(r'\d+', discount)
        discount = 1 - float(discount[0]) / 100
    except:
        discount = 1
    product = [price, discount]
    return product
    

ПЕРЕД ПАРСИНГОМ ОЗОНА ВСЕГДА ПРОВЕРЯТЬ НАЗВАНИЕ КЛАССА В КАРТОЧКЕ ТОВАРА "ЦЕНА С ОЗОН КАРТОЙ"

In [50]:
def ozon_parser(link):
    browser = webdriver.Firefox()
    browser.get(link)
    time.sleep(1)
    try:
        try:
            price = browser.find_element(By.CSS_SELECTOR, 'span[class="ol9 o7l"]').text.strip() 
        except:
            price = browser.find_element(By.CSS_SELECTOR, 'span[class="p3l lp4 l8p"]').text.strip() 
        price = price.replace('₽', '')
    except:
        price = np.nan    
    discount = 1
    product = [price, discount]
    browser.close()
    return product    

In [46]:
links = pd.read_csv('links.csv')
links = links['Ссылка'].tolist()
products = []

ДАННЫЕ ПО НАШЕМУ МАГАЗИНУ СОБИРАЮТСЯ С АКТУАЛЬНОЙ ВЫГРУЗКИ ЕКОМА, ПЕРЕД АКТУАЛЬНЫМ ПАРСИНГОМ УКАЗАТЬ АКТУАЛЬНЫЙ ПУТЬ ДО АКТУАЛЬНОГО ФАЙЛА

In [47]:
df = pd.read_csv(r'\\gold585.int\uk\Общее хранилище файлов\Дирекция Маркетинга и организации розничных продаж\Отдел аналитики\Выгрузки из БД\2023-11-16_остатки сайта.csv') # сюда 
df = df.drop_duplicates()

for link in tqdm(links):
    try:
        if 'ozon.ru' in link:
            products.append(ozon_parser(link))
        elif 'wildberries.ru' in link:
            products.append(wb_parser(link))
        elif '585zolotoy.ru' in link:
            products.append(ecom_parser(link))
        elif 'sokolov.ru' in link:
            products.append(sokolov_parser(link))
        elif 'sunlight' in link:
            products.append(sunlight_parser(link))
    except:
        products.append([np.nan, np.nan])

100%|██████████| 239/239 [12:27<00:00,  3.13s/it]


In [48]:
date = date.today()
with open(f"actual_{date}.csv", mode="w", encoding='utf-8') as file:
    file_writer = csv.writer(file, delimiter=",", lineterminator="\n")
    file_writer.writerow(['price', 'discount'])
    for i in range(0, len(products)):
        try:
            rows = products[i]
            file_writer.writerow(rows)
        except:
            pass

In [49]:
# запускать, если нужны данные в Excel формате
parser = pd.read_csv(f'actual_{date}.csv')
parser.to_excel(f'actual_{date}.xlsx')

In [ ]:
# def main(url):
#     try:
#         if 'ozon.ru' in url:
#             return ozon_parser(url)
#         elif 'wildberries.ru' in url:
#             return wb_parser(url)
#         elif '585zolotoy.ru' in url:
#             return ecom_parser(url)
#         elif 'sokolov.ru' in url:
#             return sokolov_parser(url)
#         elif 'sunlight' in url:
#             return sunlight_parser(url)
#     except:
#         return [np.nan, np.nan]

In [ ]:
# CONNECTIONS = 6

# with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
#     future_to_url = (executor.submit(main, url) for url in links)
#     for future in tqdm(concurrent.futures.as_completed(future_to_url), total=len(links)):
#         try:
#             data = future.result() 
#         except Exception as exc:      
#             data = str(type(exc))
#         finally:
#             products.append(data)

100%|██████████| 239/239 [02:56<00:00,  1.36it/s]
